In [404]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import copy
from ipywidgets import IntProgress
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [139]:
web = webdriver.Chrome("/Users/yasha_ten/Downloads/chromedriver")

In [142]:
def parse_date(players, start_date, end_date):
    start = start_date
    progress = IntProgress(min=0, max=(end_date - start_date).days, value=0)
    display(progress)
    progress1 = IntProgress(min=0, max=500, value=0)
    display(progress1)
    while (start_date < end_date):
        url = 'https://www.atptour.com/en/rankings/singles?rankDate={}&countryCode=all&rankRange=1-500&sort=rank&sortAscending=True'.format(str(start_date))
        web.get(url)
        parse_page(start_date, players, progress1)
        start_date += datetime.timedelta(7)
        progress.value = (start_date - start).days
    

In [143]:
def parse_page(date, players, progress):
    players[str(date)] = []
    names = web.find_elements_by_css_selector("td.player-cell")
    player_points = web.find_elements_by_css_selector("td.points-cell")
    if (names != []):
        for i in range(500):
            if i % 10 == 0:
                progress.value = i
            name = names[i].text
            points = int(player_points[i].text.replace(',', ''))
            rank = i + 1
            players[str(date)].append({'name': name, 'points': points, 'rank': rank})

In [657]:
def pop_empty(players):
    pl_copy = copy.deepcopy(players)
    for date in pl_copy:
        if pl_copy[date] == []:
            players.pop(date)

In [144]:
players = {}
start_date = datetime.date(2014, 1, 13)
end_date = datetime.date(2021, 1, 1)
parse_date(players, start_date, end_date)
pop_empty(players)

IntProgress(value=0, max=2545)

IntProgress(value=0, max=500)

In [658]:
def fix_names1(data):    
    players = copy.deepcopy(data)
    for date in players.keys():
        for player in players[date]:
            ## Частный случай с Кузнецовыми...
            if 'Andrey Kuznetsov' in player['name'] or 'Alvaro Lopez San Martin' in player['name']:
                if 'Andrey Kuznetsov' in player['name']:
                    player['name'] = 'Kuznetsov An.'
                else:
                    ## Частный случай с АЛВАРО...))
                    player['name'] = 'Lopez San Martin A.'
            else:
                surname = ''
                tmp = player['name'].split(' ')
                surname = tmp[-1]
                name = tmp[0][0] + '.'
                player['name'] = surname + " " + name

    return players

In [659]:
def fix_names2(data):
    players = copy.deepcopy(data)
    for date in players.keys():
        for player in players[date]:
            surname = ''
            tmp = player['name'].split(' ')
            surname = " ".join(tmp[1:])
            name = tmp[0][0] + '.'
            player['name'] = surname + " " + name

    return players

In [660]:
def fix_time(df):
    time = []
    date_fx = []
    months = {"Jan": 1, "Feb": 2, "Mar": 3,
              "Apr": 4, "May": 5, "Jun": 6,
              "Jul": 7, "Aug": 8, "Sep": 9,
              "Oct": 10, "Nov": 11, "Dec": 12
             }
    for date in df["date"]:
        mas = date.split()
        date_fx.append(datetime.date(int(mas[2][:-1]), months[mas[1]], int(mas[0])))
        podmas = mas[-1].split(":")
        time.append(datetime.time(int(podmas[0]), int(podmas[1]), 0))
    df["time"] = time
    df["date"] = date_fx

In [661]:
def fix_letters(df):
    progress = IntProgress(min=0, max=len(df), value=0)
    display(progress)
    bug_surnames = {'Hüsler M.': 'Huesler M.', 'Mcdonald M.': 'McDonald M.',
                    'del Potro J.': 'Potro J.', 'López-Pérez E.': 'Perez E.',
                   'Di Wu.': 'Wu D.', 'Yen-Hsun Lu.': 'Lu Y.', 'Mcgee J.': 'McGee J.',
                   'Zhe Li.': 'Li Z.', 'Wolf J.J.': 'Wolf J.'}
    letters = {'á': 'a', 'ã': 'a', 'ç': 'c',
           'é': 'e', 'í': 'i', 'ñ': 'n',
           'ó': 'o', 'ö': 'o', 'ú': 'u',
          'ü': 'u', 'ý': 'y', 'ć': 'c',
          'č': 'c', 'ě': 'e', 'ı': 'i',
          'ł': 'l', 'ř': 'r', 'š': 's',
          'ž': 'z'}
    for i in range(len(df)):
        p1 = df['player1'][i]
        p2 = df['player2'][i]
        if p1 in bug_surnames or p2 in bug_surnames:
            if p1 in bug_surnames:
                p1 = bug_surnames[p1]
            else:
                p2 = bug_surnames[p2]
        else:
            for c in range(len(p1)):
                if p1[c].lower() in letters:
                    if p1[c].islower():
                        p1 = p1.replace(p1[c], letters[p1[c]])
                    else:
                        p1 = p1.replace(p1[c], letters[p1[c].lower()].upper())

            for c in range(len(p2)):
                if p2[c].lower() in letters:
                    if p2[c].islower():
                        p2 = p2.replace(p2[c], letters[p2[c]])                    
                    else:
                        p2 = p2.replace(p2[c], letters[p2[c].lower()].upper())

        df['player1'][i] = p1
        df['player2'][i] = p2
        
        if i % 10 == 0:
            progress.value = i

def add_info(df, players):
    progress = IntProgress(min=0, max=len(df), value=0)
    display(progress)
    for i in range(len(df)):
        ## find required date
        date = df['date'][i]
        for d in players.keys():
            tmp = d.split("-")
            dt = datetime.date(int(tmp[0]), int(tmp[1]), int(tmp[2]))
            if (dt >= date and players[d] != []):
                break

        p1_name = df['player1'][i]
        p2_name = df['player2'][i]
        for p in players[d]:
            if p['name'] == p1_name:
                if (df['p1_points'][i] == None):
                    df['p1_points'][i] = p['points']
                    df['p1_rank'][i] = p['rank']

            if p['name'] == p2_name:
                if (df['p2_points'][i] == None):
                    df['p2_points'][i] = p['points']
                    df['p2_rank'][i] = p['rank']
        
        if i % 10 == 0:
            progress.value = i



In [634]:
df = pd.read_pickle('matches.pkl')
df = df.dropna(subset=['k1'])
df = df.dropna(subset=['k2'])
## dropping player with bug in sofascore
df = df[(df['player1'] != 'Zhang Z.') & (df['player2'] != 'Zhang Z.')]
df = df.set_index("player1").reset_index()
fix_time(df)
print(df.shape)
df.head(3)

(19511, 222)


,player1,match_url,player2,status,Surface,date,k1,k2,player1_win,round,match_duration_mnt,Location,duration_set1,duration_set2,duration_set3,duration_set4,duration_set5,aces_match_player1,aces_match_player2,double_faults_match_player1,double_faults_match_player2,first_serve_prc_match_player1,first_serve_prc_match_player2,first_serve_cnt_match_player1,first_serve_cnt_match_player2,second_serve_prc_match_player1,second_serve_prc_match_player2,second_serve_cnt_match_player1,second_serve_cnt_match_player2,points_won_match_player1,points_won_match_player2,break_points_prc_match_player1,break_points_prc_match_player2,break_points_cnt_match_player1,break_points_cnt_match_player2,first_serve_points_prc_match_player1,first_serve_points_prc_match_player2,first_serve_points_cnt_match_player1,first_serve_points_cnt_match_player2,second_serve_points_prc_match_player1,...,break_points_prc_set5_player2,break_points_cnt_set5_player1,break_points_cnt_set5_player2,first_serve_points_prc_set5_player1,first_serve_points_prc_set5_player2,first_serve_points_cnt_set5_player1,first_serve_points_cnt_set5_player2,second_serve_points_prc_set5_player1,second_serve_points_prc_set5_player2,second_serve_points_cnt_set5_player1,second_serve_points_cnt_set5_player2,max_points_in_a_row_set5_player1,max_points_in_a_row_set5_player2,receiver_points_won_set5_player1,receiver_points_won_set5_player2,score_set1_player1,score_set1_player2,score_set2_player1,score_set2_player2,score_set3_player1,score_set3_player2,score_set4_player1,score_set4_player2,score_set5_player1,score_set5_player2,tie_break_set1_player1,tie_break_set1_player2,tie_break_set2_player1,tie_break_set2_player2,tie_break_set3_player1,tie_break_set3_player2,tie_break_set4_player1,tie_break_set4_player2,tie_break_set5_player1,tie_break_set5_player2,srv_games_player1,srv_games_player2,bp_saved_cnt_player1,bp_saved_cnt_player2,time
0,Čilić M.,https://www.sofascore.com/cilic-ferrer/eNfsMhg,Ferrer D.,Ended,Clay,2015-06-01,3.40,1.33,0,(R16),112,NaN,32.0,36.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,2.0,6.0,4.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:30:00
1,Murray A.,https://www.sofascore.com/chardy-murray/BcgsEqg,Chardy J.,Ended,Clay,2015-06-01,1.08,8.00,1,(R16),173,NaN,39.0,50.0,41.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,4.0,3.0,6.0,6.0,3.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13:50:00
2,Sock J.,https://www.sofascore.com/sock-nadal/LOfswCq,Nadal R.,Ended,Clay,2015-06-01,9.00,1.07,0,(R16),173,NaN,42.0,31.0,56.0,43.0,NaN,1.0,1.0,7.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,6.0,1.0,6.0,7.0,5.0,2.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17:05:00


In [635]:
fix_letters(df)
new_players1 = fix_names1(players)
new_players2 = fix_names2(players)

IntProgress(value=0, max=19511)

In [636]:
df['p1_points'] = [None for x in range(len(df))]
df['p1_rank'] = [None for x in range(len(df))]
df['p2_points'] = [None for x in range(len(df))]
df['p2_rank'] = [None for x in range(len(df))]

## parsed ranks --> main table
add_info(df, new_players1)
add_info(df, new_players2)

## fill Nan values in rank or points (need to think over)
df['p1_points'].fillna(0, inplace=True)
df['p2_points'].fillna(0, inplace=True)
df['p1_rank'].fillna(1000, inplace=True)
df['p2_rank'].fillna(1000, inplace=True)

IntProgress(value=0, max=19511)

IntProgress(value=0, max=19511)

In [637]:
df.to_pickle('matches.pkl')

In [820]:
players_df = pd.DataFrame(players)
players_df.to_pickle('ranks.pkl')

In [821]:
## convertating parsed players' ranks to dict
def conv_to_dict(df):
    res = {}
    for key in df.keys():
        res[key] = []
        for el in df[key]:
            res[key].append(el)
    return res

In [822]:
players['2014-01-13']

[{'name': 'Rafael Nadal', 'points': 13130, 'rank': 1},
 {'name': 'Novak Djokovic', 'points': 12260, 'rank': 2},
 {'name': 'David Ferrer', 'points': 5640, 'rank': 3},
 {'name': 'Andy Murray', 'points': 5560, 'rank': 4},
 {'name': 'Juan Martin del Potro', 'points': 5415, 'rank': 5},
 {'name': 'Roger Federer', 'points': 4355, 'rank': 6},
 {'name': 'Tomas Berdych', 'points': 4180, 'rank': 7},
 {'name': 'Stan Wawrinka', 'points': 3890, 'rank': 8},
 {'name': 'Richard Gasquet', 'points': 3140, 'rank': 9},
 {'name': 'Jo-Wilfried Tsonga', 'points': 3065, 'rank': 10},
 {'name': 'Milos Raonic', 'points': 2860, 'rank': 11},
 {'name': 'Tommy Haas', 'points': 2435, 'rank': 12},
 {'name': 'John Isner', 'points': 2310, 'rank': 13},
 {'name': 'Nicolas Almagro', 'points': 2290, 'rank': 14},
 {'name': 'Mikhail Youzhny', 'points': 2145, 'rank': 15},
 {'name': 'Fabio Fognini', 'points': 1930, 'rank': 16},
 {'name': 'Kei Nishikori', 'points': 1915, 'rank': 17},
 {'name': 'Tommy Robredo', 'points': 1810, 'ra

In [823]:
import numpy as np
rows = 4
columns = 5
np.zeros([columns, rows])
i = 0
for x in range()

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

# 